Test set evaluation

In [3]:
from ast import literal_eval
import logging
import os
import random

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from datasets import Dataset
import numpy as np
import pandas as pd
import spacy
import torch
from torch.utils.data import DataLoader
import torchmetrics
from tqdm import tqdm


import sys
sys.path.append("/home/hermione/Documents/VLP/TUM/rgrg_pretrained/")
from src.dataset.constants import ANATOMICAL_REGIONS
from src.full_model.custom_collator import CustomCollator
from src.full_model.custom_dataset import CustomDataset
from src.full_model.evaluate_full_model.evaluate_language_model import (
    get_ref_sentences_for_selected_regions,
    get_generated_reports,
    compute_language_model_scores
)
from src.full_model.report_generation_model import ReportGenerationModel
from src.full_model.train_full_model import get_tokenizer
from src.path_datasets_and_weights import path_full_dataset, path_runs_full_model, path_test_set_evaluation_scores_txt_files

# specify the checkpoint you want to evaluate by setting "RUN" and "CHECKPOINT"
RUN = 20
CHECKPOINT = "checkpoint_val_loss_1.778_overall_steps_501150.pt" #src/runs/full_model/run_20/checkpoints/
BERTSCORE_SIMILARITY_THRESHOLD = 0.9
IMAGE_INPUT_SIZE = 512
BATCH_SIZE = 8
NUM_WORKERS = 10
NUM_BEAMS = 4
MAX_NUM_TOKENS_GENERATE = 300
NUM_BATCHES_OF_GENERATED_SENTENCES_TO_SAVE_TO_FILE = 100
NUM_BATCHES_OF_GENERATED_REPORTS_TO_SAVE_TO_FILE = 100

# set the seed value for reproducibility
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

"""
Folder specified by path_test_set_evaluation_scores_txt_files will have these files after test set evaluation:

    - final_scores.txt
    - generated_sentences.txt
    - generated_abnormal_sentences.txt
    - generated_reports.txt
"""
final_scores_txt_file = os.path.join(path_test_set_evaluation_scores_txt_files, "final_scores.txt")

device:  cuda:1
